# Step 3 : Modelisation

In [ ]:
import pickle 
import pandas as pd 
import numpy as np 
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

import seaborn as sns
import sklearn.metrics as metrics
import sklearn.tree as tree

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split

In [ ]:
SHARED_FOLDER = '../ressources/us_census_full'

In [ ]:
with open ('{}/train.csv'.format(SHARED_FOLDER), 'rb') as f:
    train1 = pd.read_csv(f)

with open ('{}/test.csv'.format(SHARED_FOLDER), 'rb') as f:
    test1 = pd.read_csv(f)

with open ('{}/train_cleaned.csv'.format(SHARED_FOLDER), 'rb') as f:
    train = pd.read_csv(f)

with open ('{}/test_cleaned.csv'.format(SHARED_FOLDER), 'rb') as f:
    test = pd.read_csv(f)

with open ('{}/train_income.csv'.format(SHARED_FOLDER), 'rb') as f:
    train_income = pd.read_csv(f)
    
with open ('{}/test_income.csv'.format(SHARED_FOLDER), 'rb') as f:
    target = pd.read_csv(f)

In [ ]:
del train['Unnamed: 0'] 
sns.heatmap(train.corr(), square=True)
plt.show()

In [ ]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [ ]:
train.columns

In [ ]:
del train['Unnamed: 0'] 
del test['Unnamed: 0']
del train_income['Unnamed: 0'] 
del target['Unnamed: 0']

In [ ]:
income_df = pd.DataFrame(index = train.index, columns=["income"])
income_df["income"] = train_income

target_df = pd.DataFrame(index = test.index, columns=["income"])
target_df["income"] = target

In [ ]:
all_data = pd.concat((train.loc[:,'AAGE':'AHGA_Gr'], test.loc[:,'AAGE':'AHGA_Gr']))
y = income_df.income
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y)

In [ ]:
####Random Forest###########
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train1,y_train1)

In [ ]:
y_pred = rfc.predict(X_test1)
cm = metrics.confusion_matrix(y_test1, y_pred)
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
sns.heatmap(cm, annot=True, fmt="d")
plt.ylabel("Real value")
plt.xlabel("Predicted value")
coefs = pd.Series(rfc.feature_importances_, index=X_train1.columns)
coefs.sort()
plt.subplot(2,1,2)
coefs.plot(kind="bar")
#plt.show()

In [ ]:
y_pred = rfc.predict(X_test1)
rfc_accuracy = accuracy_score(y_test1, y_pred, normalize=False)
rfc_accuracy

In [ ]:
#####xgboost model##########
import xgboost as xgb

dtrain = xgb.DMatrix(X_train1, label = y_train1)
dtest = xgb.DMatrix(X_test1, label = y_test1)

In [ ]:
###for rmse , pour avoir the best num_boost_round(where it stops)
param = {'bst:max_depth':4, 'bst:eta':0.1}
cv_xgb = xgb.cv(param, dtrain,  num_boost_round=8000, early_stopping_rounds=1000)
cv_xgb[["test-rmse-mean", "train-rmse-mean"]].plot()
plt.show()

In [ ]:
###That's for fixing paremeters of model but I cannot finish it in my ordi cause the function of GridSearchCV need much 
##time and it needs to do the same thing for every paremeter. 
from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBRegressor

param_test1 = {
 'max_depth':[3,4,5,6,7,8,9],
 'min_child_weight':[1,3,5,6]
}

gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=54, max_depth=4, min_child_weight=1, 
                                                 gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
                        param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train, y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
###scikit-learn API#####
#model_xgb = xgb.XGBClassifier()
model_xgb = xgb.XGBClassifier(colsample_bytree=0.2, gamma=0.0, max_depth=4, min_child_weight=1.5, learning_rate=0.01, reg_alpha=0.9, 
                            reg_lambda=0.6, subsample=0.2, seed=42, silent=True, objective='binary:logistic', n_estimators = 7200)
model_xgb.fit(X_train1, y_train1)

In [ ]:
y_pred = model_xgb.predict(X_test1)
cm = metrics.confusion_matrix(y_test1, y_pred)
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
sns.heatmap(cm, annot=True, fmt="d")
plt.ylabel("Real value")
plt.xlabel("Predicted value")
#plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
xgb_accuracy = accuracy_score(y_test1, y_pred, normalize=False)
xgb_accuracy

In [ ]:
############Plot features importance of XGB#######################
plt.figure(figsize=(10, 5))
xgb.plot_importance(model_xgb,max_num_features=20)
plt.tight_layout()
plt.show()

In [ ]:
######Logistic Regression###########
import sklearn.linear_model as linear_model
cls = linear_model.LogisticRegression()
cls.fit(X_train1, y_train1)

In [ ]:
y_pred = cls.predict(X_test1)
cm = metrics.confusion_matrix(y_test1, y_pred)
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
#sns.heatmap(cm, annot=True, fmt="d", xticklabels=1, yticklabels=1)
sns.heatmap(cm, annot=True, fmt="d")
plt.ylabel("Real value")
plt.xlabel("Predicted value")
coefs = pd.Series(cls.coef_[0], index=X_train1.columns)
coefs.sort()
plt.subplot(2,1,2)
coefs.plot(kind="bar")
#plt.show()

In [ ]:
y_pred = cls.predict(X_test1)
cls_accuracy = accuracy_score(y_test1, y_pred, normalize=False)
cls_accuracy

In [ ]:
#####Accuracy#######
columns = ['accuracy']
index = ['cls','rfc', 'xgb']
df = pd.DataFrame(index = index, columns = columns)
df['accuracy'].loc['cls'] = cls_accuracy
df['accuracy'].loc['rfc'] = rfc_accuracy
df['accuracy'].loc['xgb'] = xgb_accuracy
df.plot(kind="bar")
plt.show()

In [ ]:
#####Prediction######
target = target_df.income
target_pred = model_xgb.predict(X_test)
cm = metrics.confusion_matrix(target, target_pred)
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
sns.heatmap(cm, annot=True, fmt="d")
plt.ylabel("Real value")
plt.xlabel("Predicted value")
plt.show()